In [8]:
import json
import re
import ast

In [ ]:
def parse_llm_response(response):
    """
    Robustly parse JSON or Python-like structures from an LLM response.
    Returns the loaded object (dict or list), or None if parsing fails.
    """
    if not response or not isinstance(response, str):
        print('response is empty or not a string')
        print(type(response))
        return None

    # Remove markdown code fences
    response_clean = re.sub(r'```(?:json)?', '', response).strip()

    # Extract JSON or Python literal between the first { } or [ ]
    match = re.search(r'(\{.*\}|\[.*\])', response_clean, re.DOTALL)
    if not match:
        print('match is None')
        return None

    payload = match.group(1)

    # Remove trailing commas before } or ]
    payload = re.sub(r',\s*([\}\]])', r'\1', payload)
    try:
        return json.loads(payload)
    except json.JSONDecodeError:
        # Fallback to Python literal eval
        try:
            return ast.literal_eval(payload)
        except Exception:
            print('ast.literal_eval failed')
            return None

In [37]:
response ="""{
    "understanding": "Change all text in the PowerPoint to gray color across all slides.",
    "tasks": [
        {
            "page number": 1,
            "description": "Change all text in slide 1 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 1",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 2,
            "description": "Change all text in slide 2 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 2",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 3,
            "description": "Change all text in slide 3 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 3",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 4,
            "description": "Change all text in slide 4 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 4",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 5,
            "description": "Change all text in slide 5 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 5",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 6,
            "description": "Change all text in slide 6 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 6",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 7,
            "description": "Change all text in slide 7 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 7",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 8,
            "description": "Change all text in slide 8 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 8",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 9,
            "description": "Change all text in slide 9 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 9",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        },
        {
            "page number": 10,
            "description": "Change all text in slide 10 to gray",
            "target": "All text elements (title, body, shapes, text boxes, etc.) on slide 10",
            "action": "Change font color",
            "contents": {
                "color": "#808080"
            }
        }
    ]
}"""

In [38]:
parse_llm_response(response)

{'understanding': 'Change all text in the PowerPoint to gray color across all slides.',
 'tasks': [{'page number': 1,
   'description': 'Change all text in slide 1 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 1',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 2,
   'description': 'Change all text in slide 2 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 2',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 3,
   'description': 'Change all text in slide 3 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 3',
   'action': 'Change font color',
   'contents': {'color': '#808080'}},
  {'page number': 4,
   'description': 'Change all text in slide 4 to gray',
   'target': 'All text elements (title, body, shapes, text boxes, etc.) on slide 4',
   'action': 'Change font color',
   'c

In [21]:
import win32com.client
import pywintypes
import openai
from openai import OpenAI
import re

import json
import re
import ast

import re
import json
import ast
def get_simple_powerpoint_info():
    """
    현재 열려있는 PowerPoint의 페이지 수와 파일 이름만 가져옵니다.
    """
    try:
        # PowerPoint 애플리케이션에 연결
        ppt_app = win32com.client.GetObject(Class="PowerPoint.Application")
        
        # PowerPoint가 실행 중이고 열린 프레젠테이션이 있는지 확인
        if not ppt_app or not hasattr(ppt_app, 'ActivePresentation'):
            return "PowerPoint가 실행 중이 아니거나 열린 프레젠테이션이 없습니다."
        
        # 활성 프레젠테이션 가져오기
        presentation = ppt_app.ActivePresentation
        
        # 파일 이름과 페이지 수 가져오기
        file_name = presentation.Name
        slide_count = presentation.Slides.Count
        
        return {
            "파일 이름": file_name,
            "슬라이드 수": slide_count
        }
        
    except Exception as e:
        return f"오류 발생: {str(e)}"


In [24]:
get_simple_powerpoint_info()

{'파일 이름': 'math_simple.pptx', '슬라이드 수': 10}

In [10]:

def create_agent_user_prompt(page_number):
    prompt = f"""
Slide information:
- Page number: {page_number}
"""
    return prompt

In [12]:
import win32com.client
import os
import shutil

# PowerPoint 애플리케이션 시작
ppt = win32com.client.Dispatch("PowerPoint.Application")
ppt.Visible = True  # PowerPoint 창 표시

# 기존 파일 열기 또는 새 프레젠테이션 생성
# presentation = ppt.Presentations.Open(r"C:\path\to\your\presentation.pptx")
# 또는 새 프레젠테이션: 
# presentation = ppt.Presentations.Add()
# 첫 번째 슬라이드 가져오기

ppt = win32com.client.GetObject(Class="PowerPoint.Application")
presentation = ppt.ActivePresentation
slide = presentation.Slides(1)

# 예시: 텍스트 수정
shape = slide.Shapes(1)
original_text = shape.TextFrame.TextRange.Text
print(f"원본 텍스트: {original_text}")

# 텍스트 변경
shape.TextFrame.TextRange.Text = "수정된 텍스트"
print(f"변경된 텍스트: {shape.TextFrame.TextRange.Text}")

file_path = r"./presentation.pptx"
presentation.Save()
backup_path = file_path.replace('.pptx', '_backup.pptx')

# 백업 생성
# shutil.copy2(file_path, backup_path)

# # presentation = ppt.Presentations.Open(file_path)
# # slide = presentation.Slides(1)
# # shape = slide.Shapes(1)

# # # 수정
# # shape.TextFrame.TextRange.Text = "수정된 텍스트"
# # presentation.Save()

# # 복원이 필요한 경우
# presentation.Close()
# os.remove(file_path)
# shutil.move(backup_path, file_path)
presentation = ppt.Presentations.Open(file_path)

원본 텍스트: 수정된 텍스트
변경된 텍스트: 수정된 텍스트


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024894), None)

In [14]:
import win32com.client as win32
import time
import os
from datetime import datetime

ppt = win32.Dispatch("PowerPoint.Application")
ppt.Visible = True

src_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\test.pptx"
backup_dir = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\backup"

presentation = ppt.Presentations.Open(src_path)

def backup_with_timestamp(pres, backup_dir):
    os.makedirs(backup_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_path = os.path.join(backup_dir, f"backup_{ts}.pptx")
    pres.SaveCopyAs(backup_path)
    print(f"백업 저장: {backup_path}")

# 수정 1
slide = presentation.Slides(1)
shape = slide.Shapes.AddTextbox(1, 100, 100, 400, 100)
shape.TextFrame.TextRange.Text = "수정 1"
backup_with_timestamp(presentation, backup_dir)

time.sleep(1)

# 수정 2
shape.TextFrame.TextRange.Text = "수정 2"
backup_with_timestamp(presentation, backup_dir)


백업 저장: C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203444.pptx
백업 저장: C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203445.pptx


In [ ]:
import win32com.client as win32
import os

ppt = win32.Dispatch("PowerPoint.Application")
ppt.Visible = True

presentation = ppt.ActivePresentation

original_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\test.pptx"
backup_path = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\backup\backup_20260119_203446.pptx"

# 1) 백업본 생성 (원본은 그대로 두고 복사 저장)
presentation.SaveCopyAs(backup_path)

# 2) 원본 닫기 (필요시 Save 호출 여부 선택)
presentation.Close()

# 3) 방금 만든 backup.pptx 열기
backup_presentation = ppt.Presentations.Open(backup_path)

In [2]:
import win32com.client as win32

# --- 유틸: HEX -> RGB Integer 변환 ---
def _hex_to_rgb_int(hex_color: str) -> int:
    hex_color = hex_color.lstrip("#")
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    # PowerPoint RGB는 B * 65536 + G * 256 + R
    return b * 65536 + g * 256 + r



def find_shape_by_id(prs, slide_number, shape_id):
    slide = prs.Slides(slide_number)
    # 여기서는 간단히 index로 접근 (1부터 시작)
    # 실제 ID 사용 시: shape.Id 비교 루프 돌리면 됨
    try:
        shape = slide.Shapes(shape_id)
        return shape
    except Exception:
        return None
    


def set_text_style_by_range(prs, slide_number, shape_id, 
                            char_start=None, char_end=None,
                            font_size=None, color_hex=None, 
                            bold=None, italic=None, underline=None, font_name=None,
                            strike=None, subscript=None, superscript=None):
    """
    텍스트 Index를 기준으로 스타일을 수정합니다.
    char_start, char_end: 0-based 인덱스(파이썬 스타일), end는 '직전'까지.
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    if not shape or not shape.HasTextFrame:
        return f"Error: Shape {shape_id} cannot contain text."
    
    full_text_range = shape.TextFrame.TextRange
    full_text = full_text_range.Text

    # 범위를 지정하면 해당 부분만, 지정하지 않으면 전체 선택
    if char_start is not None and char_end is not None:
        length = char_end - char_start
        # PowerPoint Characters는 1-based index 사용
        target_range = full_text_range.Characters(char_start + 1, length)
    else:
        target_range = full_text_range

    # --- 스타일 제어 ---
    if font_size is not None: target_range.Font.Size = font_size
    if font_name is not None: target_range.Font.Name = font_name
    
    if bold is not None: target_range.Font.Bold = bold
    if italic is not None: target_range.Font.Italic = italic
    if underline is not None: target_range.Font.Underline = underline
    
    # 특수 효과 적용
    if strike is not None: target_range.Font.Strikethrough = strike
    if subscript is not None: target_range.Font.Subscript = subscript
    if superscript is not None: target_range.Font.Superscript = superscript
    
    if color_hex: 
        target_range.Font.Color.RGB = _hex_to_rgb_int(color_hex)
    
    return f"Successfully updated text range [{full_text[char_start:char_end]}] in Shape {shape_id}."


def main():
    # PowerPoint Application 실행
    ppt = win32.gencache.EnsureDispatch("PowerPoint.Application")
    ppt.Visible = True

    # 새 프레젠테이션 생성
    prs = ppt.Presentations.Add()

    # 기본 레이아웃(제목 + 부제목) 슬라이드 추가
    # 1: ppLayoutTitle
    slide = prs.Slides.Add(1, 1)

    # 제목 상자에 텍스트 넣기
    title_shape = slide.Shapes.Title
    title_shape.TextFrame.TextRange.Text = "Hello, SKKU PowerPoint 테스트입니다"

    # 부제목 상자에 텍스트 넣기
    subtitle_shape = slide.Shapes(2)  # 두 번째 셰이프 (부제목)
    subtitle_shape.TextFrame.TextRange.Text = "이 문장의 일부만 스타일을 바꿔보겠습니다."

    import time

    time.sleep(2)

    # 🧪 1. 제목 텍스트의 'SKKU' 부분만 빨간색 + 굵게 + 크게
    title_text = title_shape.TextFrame.TextRange.Text
    print()
    target_word = "SKKU"
    start_idx = title_text.find(target_word)
    if start_idx != -1:
        end_idx = start_idx + len(target_word)
        msg = set_text_style_by_range(
            prs,
            slide_number=1,
            shape_id=title_shape.ZOrderPosition,  # 여기서는 ZOrderPosition 사용
            char_start=start_idx,
            char_end=end_idx,
            font_size=40,
            color_hex="#FF0000",
            bold=True
        )
        print(msg)


    time.sleep(2)
    
    # 🧪 2. 부제목에서 '일부만' 텍스트를 파란색 + 이탤릭으로
    sub_text = subtitle_shape.TextFrame.TextRange.Text
    target_word2 = "일부만"
    start_idx2 = sub_text.find(target_word2)
    if start_idx2 != -1:
        end_idx2 = start_idx2 + len(target_word2)
        msg = set_text_style_by_range(
            prs,
            slide_number=1,
            shape_id=subtitle_shape.ZOrderPosition,
            char_start=start_idx2,
            char_end=end_idx2,
            font_size=24,
            color_hex="#0000FF",
            italic=True
        )
        print(msg)

    # 저장해보고 싶으면 아래 주석 해제
    # prs.SaveAs(r"C:\temp\skku_win32com_test.pptx")

if __name__ == "__main__":
    main()


Successfully updated text range [SKKU] in Shape 1.
Successfully updated text range [일부만] in Shape 2.


In [35]:
import win32com.client

def check_powerpoint_font_attributes():
    try:
        # PPT 애플리케이션 실행
        ppt = win32com.client.Dispatch("PowerPoint.Application")
        # 새 프레젠테이션과 슬라이드 추가
        prs = ppt.Presentations.Add()
        slide = prs.Slides.Add(1, 12)  # 12 = ppLayoutBlank
        
        # 텍스트 박스 하나 생성
        shape = slide.Shapes.AddTextbox(1, 100, 100, 200, 50)
        shape.TextFrame.TextRange.Text = "Check Font"
        
        # Font 객체 접근
        font = shape.TextFrame.TextRange.Font
        
        print("\n" + "="*60)
        print("현재 시스템의 PowerPoint Font 객체 속성 리스트")
        print("="*60)
        
        # 모든 속성 출력 (알파벳 순 정렬)
        attrs = sorted(dir(font))
        for attr in attrs:
            # '_'로 시작하는 내부 속성은 제외하고 출력
            print(f"    {attr}")
        print("="*60)
        
        # 리소스 정리 (저장하지 않고 종료)
        prs.Close()
        ppt.Quit()
        
    except Exception as e:
        print(f"에러 발생: {e}")

if __name__ == "__main__":
    check_powerpoint_font_attributes()


현재 시스템의 PowerPoint Font 객체 속성 리스트
    Application
    AutoRotateNumbers
    BaselineOffset
    Bold
    CLSID
    Color
    Embeddable
    Embedded
    Emboss
    Italic
    Name
    NameAscii
    NameComplexScript
    NameFarEast
    NameOther
    Parent
    Shadow
    Size
    Subscript
    Superscript
    Underline
    _ApplyTypes_
    __class__
    __delattr__
    __dict__
    __dir__
    __doc__
    __eq__
    __format__
    __ge__
    __getattr__
    __getattribute__
    __getstate__
    __gt__
    __hash__
    __init__
    __init_subclass__
    __iter__
    __le__
    __lt__
    __module__
    __ne__
    __new__
    __reduce__
    __reduce_ex__
    __repr__
    __setattr__
    __sizeof__
    __str__
    __subclasshook__
    __weakref__
    _get_good_object_
    _get_good_single_object_
    _oleobj_
    _prop_map_get_
    _prop_map_put_
    coclass_clsid


In [4]:
import win32com.client as win32

ppt = win32.gencache.EnsureDispatch("PowerPoint.Application")
ppt.Visible = True

# 새 프레젠테이션 & 슬라이드
prs = ppt.Presentations.Add()
slide = prs.Slides.Add(1, 1)  # 제목 슬라이드

shape = slide.Shapes.Title
shape.TextFrame.TextRange.Text = "이 텍스트 전체에 취소선을 적용합니다."

# 전체 TextRange
tr = shape.TextFrame.TextRange

# 취소선 켜기 (True / -1 둘 다 작동)
tr.Font.Strikethrough = True
# 또는: tr.Font.Strikethrough = -1  # msoTrue

AttributeError: '<win32com.gen_py.Microsoft PowerPoint 16.0 Object Library.Font instance at 0x2839457922000>' object has no attribute 'Strikethrough'

In [1]:
import win32com.client
import json
import re
from typing import Optional, Union, List, Dict, Any



# --- Internal Helper Functions ---

def _hex_to_rgb_int(hex_str):
    """Converts HEX string (#FFFFFF or FFFFFF) to win32-compatible BGR integer."""
    hex_str = hex_str.lstrip('#')
    if len(hex_str) != 6:
        raise ValueError("HEX code must be 6 characters long (e.g., FF0000).")
    
    r = int(hex_str[0:2], 16)
    g = int(hex_str[2:4], 16)
    b = int(hex_str[4:6], 16)
    return (b << 16) | (g << 8) | r


def find_shape_by_id(prs, slide_number, shape_id):
    """Finds a specific Shape object by its unique ID on a given slide."""
    try:
        slide = prs.Slides(slide_number)
        for shape in slide.Shapes:
            if shape.Id == shape_id:
                return shape
    except Exception as e:
        raise ValueError(f"Error accessing slide {slide_number}: {e}")
    raise ValueError(f"Shape with ID {shape_id} not found on slide {slide_number}.")


def _get_text_runs_from_shape(shape):
    if not shape.HasTextFrame or not shape.TextFrame.HasText:
        return []
    return shape.TextFrame.TextRange.Runs()


def _get_text_runs_from_table_cell(shape, row_index, col_index):
    cell = shape.Table.Cell(row_index, col_index)
    if not cell.Shape.TextFrame.HasText:
        return []
    return cell.Shape.TextFrame.TextRange.Runs()


def _resolve_runs(
    prs,
    slide_index,
    shape_id,
    container="shape",
    row_index=None,
    col_index=None
):
    shape = find_shape_by_id(prs, slide_index, shape_id)

    if container == "shape":
        return _get_text_runs_from_shape(shape)

    if container == "table_cell":
        if row_index is None or col_index is None:
            raise ValueError("row_index and col_index required for table_cell.")
        return _get_text_runs_from_table_cell(shape, row_index, col_index)

    raise ValueError(f"Unknown container type: {container}")



def _split_run_by_range(text_range, start, end):
    """
    Split a TextRange into [before][target][after] and
    return the target TextRange.
    """
    full_len = len(text_range.Text)

    if start < 0 or end > full_len or start >= end:
        raise ValueError("Invalid char range inside run.")

    # before
    if start > 0:
        text_range.Characters(1, start).InsertAfter(
            text_range.Characters(start + 1, full_len - start).Text
        )
        target = text_range.Characters(1, end - start)
    else:
        target = text_range.Characters(1, end)

    # after 제거
    if end < full_len:
        target.Characters(end - start + 1, full_len - end).Delete()

    return target


#########################################################################
######################## [A]  Text Style Editing ########################
#########################################################################
def set_text_run_style(
    prs,
    slide_index,
    shape_id,
    run_index,
    char_start=None,
    char_end=None,
    font_name=None,
    font_size=None,
    bold=None,
    italic=None,
    underline=None,
    color_hex=None
):
    """
    Modify text style at run level, with optional character-range targeting
    inside the specified run. Shape/TextBox only.
    """
    try:
        # --- resolve runs (shape only) ---
        shape = find_shape_by_id(prs, slide_index, shape_id)
        if not shape or not shape.HasTextFrame:
            raise ValueError(f"Shape {shape_id} has no text frame.")

        runs = _get_text_runs_from_shape(shape)

        if run_index < 0 or run_index >= len(runs):
            raise IndexError(f"run_index {run_index} out of range.")

        run = runs[run_index]
        tr = run.TextRange

        # --- determine target range ---
        if char_start is None and char_end is None:
            target = tr
        else:
            if char_start is None or char_end is None:
                raise ValueError("Both char_start and char_end are required.")
            target = _split_run_by_range(tr, char_start, char_end)

        # --- apply styles ---
        font = target.Font

        if font_name is not None:
            font.Name = font_name
        if font_size is not None:
            font.Size = font_size
        if bold is not None:
            font.Bold = bold
        if italic is not None:
            font.Italic = italic
        if underline is not None:
            font.Underline = underline
        if color_hex is not None:
            font.Color.RGB = _hex_to_rgb_int(color_hex)

        return (
            f"Updated style in run {run_index} "
            f"range [{char_start}:{char_end}] of Shape {shape_id}."
        )

    except Exception as e:
        return f"Error updating run style: {e}"


def set_paragraph_alignment(prs, slide_number, shape_id, alignment="left", 
                           line_spacing=None, space_before=None, space_after=None):
    """
    Adjusts paragraph-level formatting.
    
    Args:
        alignment: 'left', 'center', 'right', 'justify', 'distribute'
        line_spacing: Line spacing multiplier (e.g., 1.5)
        space_before/after: Points before/after paragraph
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    if not shape.HasTextFrame:
        return f"Error: Shape {shape_id} cannot contain text."
    
    tr = shape.TextFrame.TextRange
    alignment_map = {
        'left': 1, 'center': 2, 'right': 3, 'justify': 4, 'distribute': 5
    }
    
    if alignment in alignment_map:
        tr.ParagraphFormat.Alignment = alignment_map[alignment]
    if line_spacing:
        tr.ParagraphFormat.LineRuleWithin = 0  # Multiple spacing
        tr.ParagraphFormat.SpaceWithin = line_spacing
    if space_before is not None:
        tr.ParagraphFormat.SpaceBefore = space_before
    if space_after is not None:
        tr.ParagraphFormat.SpaceAfter = space_after
    
    return f"Paragraph formatting applied to Shape {shape_id}."


def manage_bullet_points(prs, slide_number, shape_id, bullet_type="bullet", 
                     bullet_char=None, start_value=1):
    """
    Adds or modifies bullet/numbering format. 
    
    Args:
        bullet_type: 'bullet', 'number', 'none'
        bullet_char: Custom bullet character
        start_value: Starting number for numbered lists
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    if not shape.HasTextFrame:
        return f"Error: Shape {shape_id} cannot contain text."
    
    tr = shape.TextFrame.TextRange
    
    if bullet_type == "none":
        tr.ParagraphFormat.Bullet.Visible = False
    elif bullet_type == "bullet":
        tr.ParagraphFormat.Bullet.Visible = True
        tr.ParagraphFormat.Bullet.Type = 1  # ppBulletUnnumbered
        if bullet_char:
            tr.ParagraphFormat.Bullet.Character = ord(bullet_char)
    elif bullet_type == "number":
        tr.ParagraphFormat.Bullet.Visible = True
        tr.ParagraphFormat.Bullet.Type = 2  # ppBulletNumbered
        tr.ParagraphFormat.Bullet.StartValue = start_value
    
    return f"Bullet formatting applied to Shape {shape_id}."


# --- [B] Enhanced Text Content Editing ---

# def update_text(prs, slide_number, shape_id, new_text, append=False, 
#                 paragraph_index=None):
#     """
#     Updates text content with append option and paragraph targeting.
    
#     Args:
#         append: If True, adds to existing text instead of replacing
#         paragraph_index: Target specific paragraph (0-based)
#     """
#     shape = find_shape_by_id(prs, slide_number, shape_id)
#     if not shape.HasTextFrame:
#         return f"Error: Shape {shape_id} does not support text editing."
    
#     if paragraph_index is not None:
#         para = shape.TextFrame.TextRange.Paragraphs(paragraph_index + 1)
#         if append:
#             para.Text = para.Text + new_text
#         else:
#             para.Text = new_text
#     else:
#         if append:
#             shape.TextFrame.TextRange.Text += new_text
#         else:
#             shape.TextFrame.TextRange.Text = new_text
    
#     return f"Successfully updated text for Shape {shape_id}."


def find_and_replace(prs, slide_number, shape_id, find_text, replace_text, 
                    match_case=False):
    """Finds and replaces text within a shape."""
    shape = find_shape_by_id(prs, slide_number, shape_id)
    if not shape.HasTextFrame:
        return f"Error: Shape {shape_id} does not support text editing."
    
    current_text = shape.TextFrame.TextRange.Text
    
    if match_case:
        new_text = current_text.replace(find_text, replace_text)
    else:
        # Case-insensitive replace
        new_text = re.sub(re.escape(find_text), replace_text, current_text, flags=re.IGNORECASE)
    
    shape.TextFrame.TextRange.Text = new_text
    count = len(re.findall(re.escape(find_text), current_text, re.IGNORECASE if not match_case else 0))
    
    return f"Replaced {count} occurrence(s) in Shape {shape_id}."


# --- [C] Enhanced Layout / Geometry Editing ---

def adjust_layout(prs, slide_number, shape_id, left=None, top=None, 
                 width=None, height=None, rotation=None):
    """Adjusts position, size, and rotation of a shape."""
    shape = find_shape_by_id(prs, slide_number, shape_id)
    
    if left is not None: shape.Left = left
    if top is not None: shape.Top = top
    if width is not None: shape.Width = width
    if height is not None: shape.Height = height
    if rotation is not None: shape.Rotation = rotation
    
    return f"Successfully adjusted layout for Shape {shape_id}."


def distribute_shapes(prs, slide_number, shape_ids, direction="horizontal", 
                     spacing=None):
    """
    Distributes multiple shapes evenly.
    
    Args:
        direction: 'horizontal' or 'vertical'
        spacing: Fixed spacing between shapes (if None, distribute evenly)
    """
    shapes = [find_shape_by_id(prs, slide_number, sid) for sid in shape_ids]
    
    if len(shapes) < 2:
        return "Need at least 2 shapes to distribute."
    
    if direction == "horizontal":
        shapes.sort(key=lambda s: s.Left)
        if spacing:
            for i in range(1, len(shapes)):
                shapes[i].Left = shapes[i-1].Left + shapes[i-1].Width + spacing
        else:
            total_width = sum(s.Width for s in shapes)
            start = shapes[0].Left
            end = shapes[-1].Left + shapes[-1].Width
            available = end - start - total_width
            gap = available / (len(shapes) - 1) if len(shapes) > 1 else 0
            
            current_left = start
            for shape in shapes:
                shape.Left = current_left
                current_left += shape.Width + gap
    else:  # vertical
        shapes.sort(key=lambda s: s.Top)
        if spacing:
            for i in range(1, len(shapes)):
                shapes[i].Top = shapes[i-1].Top + shapes[i-1].Height + spacing
        else:
            total_height = sum(s.Height for s in shapes)
            start = shapes[0].Top
            end = shapes[-1].Top + shapes[-1].Height
            available = end - start - total_height
            gap = available / (len(shapes) - 1) if len(shapes) > 1 else 0
            
            current_top = start
            for shape in shapes:
                shape.Top = current_top
                current_top += shape.Height + gap
    
    return f"Distributed {len(shapes)} shapes {direction}ly."


def align_shapes(prs, slide_number, shape_ids, align_type="left"):
    """
    Aligns multiple shapes to each other.
    
    Args:
        align_type: 'left', 'right', 'top', 'bottom', 'center_h', 'center_v'
    """
    shapes = [find_shape_by_id(prs, slide_number, sid) for sid in shape_ids]
    
    if len(shapes) < 2:
        return "Need at least 2 shapes to align."
    
    if align_type == "left":
        left_most = min(s.Left for s in shapes)
        for shape in shapes:
            shape.Left = left_most
    elif align_type == "right":
        right_most = max(s.Left + s.Width for s in shapes)
        for shape in shapes:
            shape.Left = right_most - shape.Width
    elif align_type == "top":
        top_most = min(s.Top for s in shapes)
        for shape in shapes:
            shape.Top = top_most
    elif align_type == "bottom":
        bottom_most = max(s.Top + s.Height for s in shapes)
        for shape in shapes:
            shape.Top = bottom_most - shape.Height
    elif align_type == "center_h":
        avg_center = sum(s.Left + s.Width / 2 for s in shapes) / len(shapes)
        for shape in shapes:
            shape.Left = avg_center - shape.Width / 2
    elif align_type == "center_v":
        avg_center = sum(s.Top + s.Height / 2 for s in shapes) / len(shapes)
        for shape in shapes:
            shape.Top = avg_center - shape.Height / 2
    
    return f"Aligned {len(shapes)} shapes by {align_type}."


# --- [D] Enhanced Object Lifecycle ---

def manage_object(prs, slide_number, action, shape_id=None, shape_type=1, 
                 left=100, top=100, width=100, height=100, text=None):
    """Creates, deletes, or duplicates shapes."""
    slide = prs.Slides(slide_number)
    
    if action == "add":
        new_shape = slide.Shapes.AddShape(shape_type, left, top, width, height)
        if text and new_shape.HasTextFrame:
            new_shape.TextFrame.TextRange.Text = text
        return f"Shape created successfully (ID: {new_shape.Id})."
    
    elif action == "delete" and shape_id:
        find_shape_by_id(prs, slide_number, shape_id).Delete()
        return f"Shape {shape_id} deleted successfully."
    
    elif action == "duplicate" and shape_id:
        original = find_shape_by_id(prs, slide_number, shape_id)
        duplicate = original.Duplicate()
        duplicate.Left += 20  # Offset slightly
        duplicate.Top += 20
        return f"Shape {shape_id} duplicated (New ID: {duplicate.Id})."
    
    return "Invalid action or missing shape_id."


def add_textbox(prs, slide_number, left, top, width, height, text=""):
    """Creates a new textbox with specified text."""
    slide = prs.Slides(slide_number)
    textbox = slide.Shapes.AddTextbox(1, left, top, width, height)  # msoTextOrientationHorizontal
    if text:
        textbox.TextFrame.TextRange.Text = text
    return f"Textbox created (ID: {textbox.Id})."


def add_image(prs, slide_number, image_path, left, top, width=None, height=None):
    """Inserts an image onto the slide."""
    slide = prs.Slides(slide_number)
    
    if width and height:
        picture = slide.Shapes.AddPicture(image_path, False, True, left, top, width, height)
    else:
        picture = slide.Shapes.AddPicture(image_path, False, True, left, top)
    
    return f"Image inserted (ID: {picture.Id})."


def group_shapes(prs, slide_number, shape_ids):
    """Groups multiple shapes together."""
    slide = prs.Slides(slide_number)
    shapes = [find_shape_by_id(prs, slide_number, sid) for sid in shape_ids]
    
    # Create shape range
    shape_range = slide.Shapes.Range([s.Id for s in shapes])
    grouped = shape_range.Group()
    
    return f"Grouped {len(shape_ids)} shapes (Group ID: {grouped.Id})."


def ungroup_shapes(prs, slide_number, group_id):
    """Ungroups a grouped shape."""
    group = find_shape_by_id(prs, slide_number, group_id)
    ungrouped = group.Ungroup()
    
    return f"Ungrouped shape {group_id} into {ungrouped.Count} shapes."


# --- [E] Enhanced Visual Style / Theme ---

def apply_visual_style(prs, slide_number, shape_id, bg_color_hex=None, 
                      line_color_hex=None, line_weight=None, line_style=None,
                      transparency=None, shadow=None):
    """
    Sets comprehensive visual styles.
    
    Args:
        line_style: 'solid', 'dash', 'dot', 'dash_dot'
        transparency: 0-1 (0=opaque, 1=transparent)
        shadow: True/False to enable/disable shadow
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    results = []

    if bg_color_hex:
        shape.Fill.Visible = True 
        shape.Fill.ForeColor.RGB = _hex_to_rgb_int(bg_color_hex)
        results.append(f"background({bg_color_hex})")

    if transparency is not None:
        shape.Fill.Transparency = transparency
        results.append(f"transparency({transparency})")

    if line_color_hex:
        shape.Line.Visible = True 
        shape.Line.ForeColor.RGB = _hex_to_rgb_int(line_color_hex)
        results.append(f"line color({line_color_hex})")

    if line_weight is not None:
        shape.Line.Visible = True
        shape.Line.Weight = line_weight
        results.append(f"line weight({line_weight}pt)")
    
    if line_style:
        shape.Line.Visible = True
        style_map = {'solid': 1, 'dash': 2, 'dot': 3, 'dash_dot': 4}
        if line_style in style_map:
            shape.Line.DashStyle = style_map[line_style]
            results.append(f"line style({line_style})")
    
    if shadow is not None:
        shape.Shadow.Visible = shadow
        results.append(f"shadow({'on' if shadow else 'off'})")

    return f"Shape {shape_id} visual style updated: " + ", ".join(results) if results else f"No changes applied to Shape {shape_id}."


def apply_gradient_fill(prs, slide_number, shape_id, color1_hex, color2_hex, 
                       gradient_type="linear", angle=0):
    """
    Applies gradient fill to a shape.
    
    Args:
        gradient_type: 'linear', 'radial', 'rectangular', 'path'
        angle: Gradient angle in degrees (for linear)
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    
    gradient_map = {'linear': 1, 'radial': 3, 'rectangular': 4, 'path': 5}
    
    shape.Fill.TwoColorGradient(gradient_map.get(gradient_type, 1), 1)
    shape.Fill.ForeColor.RGB = _hex_to_rgb_int(color1_hex)
    shape.Fill.BackColor.RGB = _hex_to_rgb_int(color2_hex)
    
    if gradient_type == "linear":
        # Set gradient angle
        shape.Fill.GradientAngle = angle
    
    return f"Gradient applied to Shape {shape_id}."


def set_shape_effect(prs, slide_number, shape_id, effect_type, **kwargs):
    """
    Applies special effects to shapes.
    
    Args:
        effect_type: 'glow', 'soft_edge', 'reflection', '3d'
        kwargs: Effect-specific parameters
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    
    if effect_type == "glow":
        color_hex = kwargs.get('color_hex', 'FFFF00')
        size = kwargs.get('size', 10)
        shape.Glow.Color.RGB = _hex_to_rgb_int(color_hex)
        shape.Glow.Radius = size
        return f"Glow effect applied to Shape {shape_id}."
    
    elif effect_type == "soft_edge":
        radius = kwargs.get('radius', 5)
        shape.SoftEdge.Radius = radius
        return f"Soft edge applied to Shape {shape_id}."
    
    elif effect_type == "reflection":
        shape.Reflection.Type = 1  # Enable reflection
        return f"Reflection applied to Shape {shape_id}."
    
    return f"Unknown effect type: {effect_type}"


# --- [F] Enhanced Consistency / Polishing ---

def align_to_object(prs, slide_number, target_id, base_id, side="right", margin=10):
    """Aligns the target shape relative to a base shape with custom margin."""
    target = find_shape_by_id(prs, slide_number, target_id)
    base = find_shape_by_id(prs, slide_number, base_id)
    
    if side == "right":
        target.Left = base.Left + base.Width + margin
        target.Top = base.Top
    elif side == "left":
        target.Left = base.Left - target.Width - margin
        target.Top = base.Top
    elif side == "bottom":
        target.Left = base.Left
        target.Top = base.Top + base.Height + margin
    elif side == "top":
        target.Left = base.Left
        target.Top = base.Top - target.Height - margin
    elif side == "center":
        target.Left = base.Left + (base.Width - target.Width) / 2
        target.Top = base.Top + (base.Height - target.Height) / 2
        
    return f"Aligned {target_id} to the {side} of {base_id}."


def match_formatting(prs, slide_number, source_id, target_ids):
    """Copies formatting from source shape to target shapes."""
    source = find_shape_by_id(prs, slide_number, source_id)
    targets = [find_shape_by_id(prs, slide_number, tid) for tid in target_ids]
    
    for target in targets:
        # Copy fill
        if source.Fill.Visible:
            target.Fill.ForeColor.RGB = source.Fill.ForeColor.RGB
            target.Fill.Transparency = source.Fill.Transparency
        
        # Copy line
        if source.Line.Visible:
            target.Line.ForeColor.RGB = source.Line.ForeColor.RGB
            target.Line.Weight = source.Line.Weight
        
        # Copy text format if applicable
        if source.HasTextFrame and target.HasTextFrame:
            src_tr = source.TextFrame.TextRange
            tgt_tr = target.TextFrame.TextRange
            tgt_tr.Font.Name = src_tr.Font.Name
            tgt_tr.Font.Size = src_tr.Font.Size
            tgt_tr.Font.Color.RGB = src_tr.Font.Color.RGB
            tgt_tr.Font.Bold = src_tr.Font.Bold
            tgt_tr.Font.Italic = src_tr.Font.Italic
    
    return f"Formatting copied from {source_id} to {len(target_ids)} shape(s)."


def set_z_order(prs, slide_number, shape_id, order="bring_to_front"):
    """
    Changes the z-order (layering) of a shape.
    
    Args:
        order: 'bring_to_front', 'send_to_back', 'bring_forward', 'send_backward'
    """
    shape = find_shape_by_id(prs, slide_number, shape_id)
    
    if order == "bring_to_front":
        shape.ZOrder(0)  # msoBringToFront
    elif order == "send_to_back":
        shape.ZOrder(1)  # msoSendToBack
    elif order == "bring_forward":
        shape.ZOrder(2)  # msoBringForward
    elif order == "send_backward":
        shape.ZOrder(3)  # msoSendBackward
    
    return f"Z-order changed for Shape {shape_id}."


# --- [G] Slide Management ---

def add_slide(prs, layout_index=1, position=None):
    """
    Adds a new slide to the presentation.
    
    Args:
        layout_index: Layout to use (1-based)
        position: Where to insert (None = end)
    """
    layout = prs.SlideMaster.CustomLayouts(layout_index)
    
    if position:
        new_slide = prs.Slides.AddSlide(position, layout)
    else:
        new_slide = prs.Slides.Add(prs.Slides.Count + 1, layout)
    
    return f"Slide added at position {new_slide.SlideIndex}."


def delete_slide(prs, slide_number):
    """Deletes a specific slide."""
    prs.Slides(slide_number).Delete()
    return f"Slide {slide_number} deleted."


def duplicate_slide(prs, slide_number):
    """Duplicates a specific slide."""
    original = prs.Slides(slide_number)
    duplicate = original.Duplicate()
    return f"Slide {slide_number} duplicated to position {duplicate.SlideIndex}."


# --- [H] Table Operations ---

def add_table(prs, slide_number, rows, cols, left, top, width, height):
    """Creates a table on the slide."""
    slide = prs.Slides(slide_number)
    table = slide.Shapes.AddTable(rows, cols, left, top, width, height)
    return f"Table created (ID: {table.Id}, {rows}x{cols})."


def update_table_cell(prs, slide_number, table_id, row, col, text, 
                     font_size=None, color_hex=None, bg_color_hex=None):
    """Updates content and style of a specific table cell."""
    table_shape = find_shape_by_id(prs, slide_number, table_id)
    
    if not table_shape.HasTable:
        return f"Error: Shape {table_id} is not a table."
    
    cell = table_shape.Table.Cell(row, col)
    cell.Shape.TextFrame.TextRange.Text = text
    
    if font_size:
        cell.Shape.TextFrame.TextRange.Font.Size = font_size
    if color_hex:
        cell.Shape.TextFrame.TextRange.Font.Color.RGB = _hex_to_rgb_int(color_hex)
    if bg_color_hex:
        cell.Shape.Fill.ForeColor.RGB = _hex_to_rgb_int(bg_color_hex)
    
    return f"Table cell ({row},{col}) updated."


# --- [I] Animation & Transition ---

def add_animation(prs, slide_number, shape_id, effect_type="appear", 
                 trigger="on_click", duration=1.0):
    """
    Adds animation to a shape.
    
    Args:
        effect_type: 'appear', 'fade', 'fly_in', 'zoom', etc.
        trigger: 'on_click', 'with_previous', 'after_previous'
        duration: Animation duration in seconds
    """
    slide = prs.Slides(slide_number)
    shape = find_shape_by_id(prs, slide_number, shape_id)
    
    effect_map = {
        'appear': 1,  # msoAnimEffectAppear
        'fade': 10,   # msoAnimEffectFade
        'fly_in': 22, # msoAnimEffectFly
        'zoom': 88,   # msoAnimEffectZoom
    }
    
    effect = slide.TimeLine.MainSequence.AddEffect(
        shape, effect_map.get(effect_type, 1), trigger=1, index=-1
    )
    effect.Timing.Duration = duration
    
    return f"Animation '{effect_type}' added to Shape {shape_id}."


def set_slide_transition(prs, slide_number, transition_type="fade", 
                        duration=1.0, advance_on_time=None):
    """
    Sets slide transition effect.
    
    Args:
        transition_type: 'fade', 'push', 'wipe', 'split', etc.
        duration: Transition duration in seconds
        advance_on_time: Auto-advance after N seconds (None = manual)
    """
    slide = prs.Slides(slide_number)
    
    transition_map = {
        'fade': 1,
        'push': 13,
        'wipe': 15,
        'split': 14,
    }
    
    slide.SlideShowTransition.EntryEffect = transition_map.get(transition_type, 1)
    slide.SlideShowTransition.Duration = duration
    
    if advance_on_time:
        slide.SlideShowTransition.AdvanceOnTime = True
        slide.SlideShowTransition.AdvanceTime = advance_on_time
    else:
        slide.SlideShowTransition.AdvanceOnClick = True
    
    return f"Transition '{transition_type}' applied to slide {slide_number}."


FUNCTION_MAP = {
    # [A]  Text Style Editing 
    "set_text_run_style": set_text_run_style,
    "set_paragraph_alignment": set_paragraph_alignment,
    "manage_bullet_points": manage_bullet_points,


    "find_and_replace": find_and_replace,
    "adjust_layout": adjust_layout,
    "distribute_shapes": distribute_shapes,
    "align_shapes": align_shapes,
    "manage_object": manage_object,
    "add_textbox": add_textbox,
    "apply_visual_style": apply_visual_style,
    "apply_gradient_fill": apply_gradient_fill,
    "add_slide": add_slide,
    "delete_slide": delete_slide,
    "duplicate_slide": duplicate_slide
}




In [9]:
set_text_run_style(**{
    'prs': prs,
    'slide_index': 1,
    'shape_id': 2,
    'run_index': 0,
    'char_start': 0,
    'char_end': 24,
    'font_name': 'Calibri',
    'font_size': 60,
    'bold': False,
    'italic': False,
    'underline': False,
    'color_hex': '#FF0000'
})

"Error updating run style: 'TextRange' object is not subscriptable"

In [ ]:
import os
import json
import time
import win32com.client
from loguru import logger
import subprocess

from editppt.utils.logtime import *
from agent import *
from parser import Parser
from planner import Planner
from editppt.utils.utils import *

import sys

ppt_app = win32com.client.GetActiveObject("PowerPoint.Application")
prs = ppt_app.ActivePresentation

In [2]:
len("""Scalar: magnitude only
Vector: magnitude + direction (boldface, lowercase)
Matrix: 2D array of numbers (uppercase)



""")

118

In [29]:
import win32com.client

ppt = win32com.client.Dispatch("PowerPoint.Application")
ppt.Visible = True

presentation = ppt.Presentations.Add()
slide = presentation.Slides.Add(1, 12)

textbox = slide.Shapes.AddTextbox(1, 100, 100, 600, 200)
tr = textbox.TextFrame.TextRange

# =========================
# 1. Strikethrough
# =========================
# tr.Text = "Strikethrough text\n"
# strike = tr.Characters(1, len("Strikethrough text"))
# strike.Font.Strikethrough = -1   # ← 핵심 수정

# =========================
# 2. Subscript
# =========================
start = len(tr.Text) + 1
tr.InsertAfter("H2O (Subscript)\n")
sub = tr.Characters(start + 1, 1)  # '2'
sub.Font.Subscript = -1

# =========================
# 3. Superscript
# =========================
start = len(tr.Text) + 1
tr.InsertAfter("x2 (Superscript)")
sup = tr.Characters(start + 1, 1)  # '2'
sup.Font.Superscript = -1

In [33]:
import win32com.client

# PowerPoint 실행
ppt = win32com.client.Dispatch("PowerPoint.Application")
ppt.Visible = True

# 프레젠테이션 생성
presentation = ppt.Presentations.Add()

# 빈 슬라이드 (12 = ppLayoutBlank)
slide = presentation.Slides.Add(1, 12)

# =========================
# 차트 추가
# =========================
# 51 = xlColumnClustered
chart_shape = slide.Shapes.AddChart(51, 100, 100, 500, 300)
chart = chart_shape.Chart

# 차트 제목 활성화
chart.HasTitle = True
chart.ChartTitle.Text = "This text is strikethrough"

# =========================
# 핵심: TextFrame2 + Font2
# =========================
text_range2 = chart.ChartTitle.Format.TextFrame2.TextRange

# Strikethrough 적용
text_range2.Font.StrikeThrough = -1  # msoTrue

In [26]:
import win32com.client


def create_test_ppt():
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True

    prs = ppt.Presentations.Add()
    slide = prs.Slides.Add(1, 1)  # 1 = ppLayoutText

    return ppt, prs, slide



def add_test_textbox(slide):
    # 위치 / 크기
    left, top, width, height = 100, 150, 500, 100

    shape = slide.Shapes.AddTextbox(
        Orientation=1,  # msoTextOrientationHorizontal
        Left=left,
        Top=top,
        Width=width,
        Height=height,
    )

    tr = shape.TextFrame.TextRange

    # 전체 텍스트
    tr.Text = "이름: 홍길동\n직책: 개발자"

    # "이름:" 스타일
    r1 = tr.Characters(1, 3)
    r1.Font.Bold = True
    r1.Font.Italic = True
    r1.Font.Size = 48

    # "홍길동" 스타일
    r2 = tr.Characters(5, 3)
    r2.Font.Color.RGB = 255  # Red
    r2.Font.Size = 20

    # "직책:" 스타일
    r3 = tr.Characters(9, 3)
    # r3.Font.Bold = True
    r3.Font.Size = 18

    return shape.Id


def add_rich_test_textbox(slide):
    shape = slide.Shapes.AddTextbox(
        Orientation=1,
        Left=100,
        Top=120,
        Width=650,
        Height=220,
    )

    tr = shape.TextFrame.TextRange

    tr.Text = (
        "이름: 홍길동 (Hong Gil-dong)\n"
        "직책: Senior Developer\n"
        "EMAIL: hong@example.com\n"
        "Status: ACTIVE"
    )

    # "이름:" → Bold + Big
    r = tr.Characters(1, 3)
    r.Font.Bold = True
    r.Font.Size = 26

    # "홍길동" → Red
    r = tr.Characters(5, 3)
    r.Font.Color.RGB = 255

    # "(Hong Gil-dong)" → Italic + Gray
    r = tr.Characters(9, 15)
    r.Font.Italic = True
    r.Font.Color.RGB = 8421504  # Gray

    # "직책:" → Bold
    r = tr.Characters(25, 3)
    r.Font.Bold = True

    # "Senior Developer" → Blue + Size down
    r = tr.Characters(29, 16)
    r.Font.Color.RGB = 16711680  # Blue
    r.Font.Size = 16

    # "EMAIL:" → Uppercase + Bold
    r = tr.Characters(46, 6)
    r.Font.Bold = True
    r.Font.Size = 18

    # email address → Underline
    r = tr.Characters(53, 16)
    r.Font.Underline = True

    # "Status:" → Bold
    r = tr.Characters(70, 7)
    r.Font.Bold = True

    # "ACTIVE" → Green
    r = tr.Characters(78, 6)
    r.Font.Color.RGB = 32768  # Green

    return shape.Id


In [45]:
import win32com.client


def open_ppt(path: str):
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(path)
    return ppt, presentation


def find_shape_by_id(prs, slide_number: int, shape_id: int):
    slide = prs.Slides(slide_number)
    for shape in slide.Shapes:
        if shape.Id == shape_id:
            return shape
    raise ValueError(f"Shape {shape_id} not found")


def get_text_range(prs, slide_number, shape_id):
    shape = find_shape_by_id(prs, slide_number, shape_id)
    if not shape.HasTextFrame or not shape.TextFrame.HasText:
        raise ValueError("Shape has no text")
    return shape.TextFrame.TextRange


def get_text(prs, slide_number, shape_id):
    return get_text_range(prs, slide_number, shape_id).Text

def _get_text_with_offsets(
    prs,
    slide_number: int,
    shape_id: int,
    *,
    container: str = "shape",
    row_index: int = None,
    col_index: int = None,
):
    shape = find_shape_by_id(prs, slide_number, shape_id)

    if container == "shape":
        if not shape.HasTextFrame or not shape.TextFrame.HasText:
            return "", []
        tr = shape.TextFrame.TextRange

    elif container == "table_cell":
        if row_index is None or col_index is None:
            raise ValueError("row_index / col_index required.")
        cell = shape.Table.Cell(row_index, col_index)
        if not cell.Shape.TextFrame.HasText:
            return "", []
        tr = cell.Shape.TextFrame.TextRange

    else:
        raise ValueError(f"Unknown container: {container}")

    text = tr.Text or ""
    return text, list(range(len(text)))


def _normalize_char_range(
    text: str,
    char_start: int,
    target_text: str,
    char_end: int = None,
):
    if char_start < 0 or char_start >= len(text):
        raise ValueError("char_start out of range.")

    expected_len = len(target_text)
    primary_end = char_start + expected_len

    if text[char_start:primary_end] == target_text:
        return char_start, primary_end

    # If llm gave end
    if char_end is not None:
        if text[char_start:char_end] == target_text:
            return char_start, char_end

    # Window search
    window_start = max(0, char_start - 5)
    window_end = min(len(text), char_start + expected_len + 5)
    window = text[window_start:window_end]

    idx = window.find(target_text)
    if idx != -1:
        start = window_start + idx
        return start, start + expected_len

    raise ValueError("Unable to resolve exact character range.")



In [46]:
def _normalize_char_range(text, char_start, target_text, char_end=None):
    expected_len = len(target_text)
    primary_end = char_start + expected_len

    if text[char_start:primary_end] == target_text:
        return char_start, primary_end

    if char_end is not None:
        if text[char_start:char_end] == target_text:
            return char_start, char_end

    window_start = max(0, char_start - 5)
    window_end = min(len(text), char_start + expected_len + 5)
    window = text[window_start:window_end]

    idx = window.find(target_text)
    if idx != -1:
        start = window_start + idx
        return start, start + expected_len

    raise ValueError("Unable to resolve exact range")


def _resolve_insert_position(text, preceding_text, approx_index):
    if preceding_text:
        idx = text.find(preceding_text)
        if idx != -1:
            return idx + len(preceding_text)

    return max(0, min(len(text), approx_index))


In [47]:
def insert_text_from_textbox(
    prs,
    slide_number,
    shape_id,
    preceding_text,
    approx_index,
    new_text,
):
    text = get_text(prs, slide_number, shape_id)
    insert_pos = _resolve_insert_position(
        text, preceding_text, approx_index
    )

    tr = get_text_range(prs, slide_number, shape_id)

    # PowerPoint is 1-based
    anchor = tr.Characters(insert_pos + 1, 0)
    anchor.InsertAfter(new_text)

    print(f"[INSERT] pos={insert_pos}, text='{new_text}'")

def _capture_font_style(char_range):
    font = char_range.Font
    return {
        "Name": font.Name,
        "Size": font.Size,
        "Bold": font.Bold,
        "Italic": font.Italic,
        "Underline": font.Underline,
        "Color": font.Color.RGB,
    }


def _apply_font_style(char_range, style):
    font = char_range.Font
    font.Name = style["Name"]
    font.Size = style["Size"]
    font.Bold = style["Bold"]
    font.Italic = style["Italic"]
    font.Underline = style["Underline"]
    font.Color.RGB = style["Color"]


def replace_text_from_textbox(
    prs,
    slide_number,
    shape_id,
    target_text,
    approx_start,
    new_text,
    *,
    container="shape",
    row_index=None,
    col_index=None,
):
    text, _ = _get_text_with_offsets(
        prs, slide_number, shape_id,
        container=container,
        row_index=row_index,
        col_index=col_index,
    )

    start, end = _normalize_char_range(
        text=text,
        char_start=approx_start,
        target_text=target_text,
    )

    shape = find_shape_by_id(prs, slide_number, shape_id)
    tr = (
        shape.TextFrame.TextRange
        if container == "shape"
        else shape.Table.Cell(row_index, col_index).Shape.TextFrame.TextRange
    )

    # 1️⃣ 기존 텍스트 range 확보
    old_range = tr.Characters(start + 1, end - start)

    # 2️⃣ ⭐ 스타일 먼저 캡쳐 (이게 핵심)
    style = _capture_font_style(old_range)

    # 3️⃣ 텍스트 교체 (Delete + Insert 대신 Text 치환)
    old_range.Text = new_text

    # 4️⃣ 새 텍스트 range 재획득
    new_range = tr.Characters(start + 1, len(new_text))

    # 5️⃣ ⭐ 스타일 명시 적용 (절대 안 깨짐)
    _apply_font_style(new_range, style)

    return {
        "operation": "replace",
        "range": [start, start + len(new_text)],
        "original_text": target_text,
        "new_text": new_text,
        "shape_id": shape_id,
        "slide": slide_number,
    }

def delete_text_from_textbox(
    prs,
    slide_number,
    shape_id,
    target_text,
    approx_start,
):
    text = get_text(prs, slide_number, shape_id)
    start, end = _normalize_char_range(
        text, approx_start, target_text
    )

    tr = get_text_range(prs, slide_number, shape_id)
    tr.Characters(start + 1, end - start).Delete()

    print(f"[DELETE] range=({start}, {end}), text='{target_text}'")


In [50]:
ppt, prs, slide = create_test_ppt()
shape_id = add_rich_test_textbox(slide)

In [51]:
replace_text_from_textbox(
    prs, 1, shape_id,
    target_text="이름:",
    approx_start=0,
    new_text="Name:"
)

{'operation': 'replace',
 'range': [0, 5],
 'original_text': '이름:',
 'new_text': 'Name:',
 'shape_id': 4,
 'slide': 1}

In [52]:
replace_text_from_textbox(
    prs, 1, shape_id,
    target_text="홍길동",
    approx_start=4,
    new_text="Hong Gildong"
)


{'operation': 'replace',
 'range': [6, 18],
 'original_text': '홍길동',
 'new_text': 'Hong Gildong',
 'shape_id': 4,
 'slide': 1}

In [53]:
insert_text_from_textbox(
    prs, 1, shape_id,
    preceding_text="Developer",
    approx_index=0,
    new_text=" (Backend)"
)


[INSERT] pos=55, text=' (Backend)'


In [56]:
delete_text_from_textbox(
    prs, 1, shape_id,
    target_text="EMAIL:",
    approx_start=40
)

ValueError: Unable to resolve exact range

In [35]:
replace_text_from_textbox(
    prs, 1, shape_id,
    target_text="Status:",
    approx_start=65,
    new_text="State:"
)


ValueError: Unable to resolve exact range

In [80]:
import win32com.client

def create_test_ppt_with_runs():
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True

    prs = ppt.Presentations.Add()
    slide = prs.Slides.Add(1, 2)  # ppLayoutText

    shape = slide.Shapes.Placeholders(2)
    tf = shape.TextFrame
    # tf.Clear()

    tr = tf.TextRange

    # ---- 테스트 텍스트 구성 ----
    tr.Text = "Scalar"
    tr.Font.Bold = True

    tr.InsertAfter(": ")
    tr.Characters(7, 1).Font.Bold = False

    tr.InsertAfter("magnitude only\r")
    tr.Characters(9, 14).Font.Bold = False

    start = len(tr.Text) + 1
    tr.InsertAfter("Vector")
    tr.Characters(start, 6).Font.Bold = True
    tr.Characters(start, 6).Font.Italic = True

    tr.InsertAfter(": magnitude + direction\r")

    start = len(tr.Text) + 1
    tr.InsertAfter("Matrix")
    tr.Characters(start, 6).Font.Subscript = True

    tr.InsertAfter(": 2D array of numbers")

    return ppt, prs, slide, shape

def snap_font(font):
    return (
        font.Name,
        font.Size,
        font.Bold,
        font.Italic,
        font.Underline,
        font.Color.RGB,
    )


def make_run_dict(char_range, run_start):
    font = char_range.Font
    return {
        "Text": char_range.Text,
        "Run_Start_Index": run_start,
        "Font": {
            "Name": font.Name,
            "Size": font.Size,
            "Bold": bool(font.Bold),
            "Italic": bool(font.Italic),
            "Underline": bool(font.Underline),
            "Color": font.Color.RGB,
        },
    }


def parse_runs_with_start_index(tr):
    full = tr.Text
    n = len(full)

    runs = []
    cur_idx = 1
    cur_snap = snap_font(tr.Characters(cur_idx, 1).Font)

    for i in range(2, n + 1):
        nxt_snap = snap_font(tr.Characters(i, 1).Font)
        if nxt_snap != cur_snap:
            seg_len = i - cur_idx
            run_start = cur_idx - 1
            runs.append(make_run_dict(tr.Characters(cur_idx, seg_len), run_start))
            cur_idx = i
            cur_snap = nxt_snap

    last_len = n - cur_idx + 1
    if last_len > 0:
        run_start = cur_idx - 1
        runs.append(make_run_dict(tr.Characters(cur_idx, last_len), run_start))

    return runs



def snap_font(font):
    return (
        font.Name,
        font.Size,
        font.Bold,
        font.Italic,
        font.Underline,
        font.Color.RGB,
    )


def make_run_dict(char_range, run_start):
    font = char_range.Font
    return {
        "Text": char_range.Text,
        "Run_Start_Index": run_start,
        "Font": {
            "Name": font.Name,
            "Size": font.Size,
            "Bold": bool(font.Bold),
            "Italic": bool(font.Italic),
            "Underline": bool(font.Underline),
            "Color": font.Color.RGB,
        },
    }


def parse_runs_with_start_index(tr):
    full = tr.Text
    n = len(full)

    runs = []
    cur_idx = 1
    cur_snap = snap_font(tr.Characters(cur_idx, 1).Font)

    for i in range(2, n + 1):
        nxt_snap = snap_font(tr.Characters(i, 1).Font)
        if nxt_snap != cur_snap:
            seg_len = i - cur_idx
            run_start = cur_idx - 1
            runs.append(make_run_dict(tr.Characters(cur_idx, seg_len), run_start))
            cur_idx = i
            cur_snap = nxt_snap

    last_len = n - cur_idx + 1
    if last_len > 0:
        run_start = cur_idx - 1
        runs.append(make_run_dict(tr.Characters(cur_idx, last_len), run_start))

    return runs





def _hex_to_rgb_int(hex_str):
    """Converts HEX string (#FFFFFF or FFFFFF) to win32-compatible BGR integer."""
    hex_str = hex_str.lstrip('#')
    if len(hex_str) != 6:
        raise ValueError("HEX code must be 6 characters long (e.g., FF0000).")
    
    r = int(hex_str[0:2], 16)
    g = int(hex_str[2:4], 16)
    b = int(hex_str[4:6], 16)
    return (b << 16) | (g << 8) | r


def iter_run_slices_from_json(runs, target_start, target_end):
    """
    JSON Runs → (slice_start, slice_end, run)
    """
    for run in runs:
        rs = run["Run_Start_Index"]
        re = rs + len(run["Text"])

        if re <= target_start:
            continue
        if rs >= target_end:
            break

        yield max(rs, target_start), min(re, target_end), run

        
def apply_color_run_preserve(tr, run_slices, color):
    if isinstance(color, str):
        rgb = _hex_to_rgb_int(color)
    elif isinstance(color, int):
        rgb = _hex_to_rgb_int(color)
    else:
        raise TypeError("color must be hex string or RGB int")

    for start, end, _ in run_slices:
        length = end - start
        if length <= 0:
            continue
        tr.Characters(start + 1, length).Font.Color.RGB = rgb



AttributeError: 'int' object has no attribute 'lstrip'

In [87]:
sentences = [
    "나는 밥을 먹었다.",
    "我吃了饭。",
    "I ate a meal.",
    "Comí la comida.",
    "मैंने खाना खाया।",
    "أكلتُ الطعامَ.",
    "J'ai mangé.",
    "Eu comi.",
    "Я поел.",
    "ご飯を食べました。",
    "Ich habe gegessen."
]

for s in sentences:
    print(repr(s), "=> 길이:", len(s))

'나는 밥을 먹었다.' => 길이: 10
'我吃了饭。' => 길이: 5
'I ate a meal.' => 길이: 13
'Comí la comida.' => 길이: 15
'मैंने खाना खाया।' => 길이: 16
'أكلتُ الطعامَ.' => 길이: 14
"J'ai mangé." => 길이: 11
'Eu comi.' => 길이: 8
'Я поел.' => 길이: 7
'ご飯を食べました。' => 길이: 9
'Ich habe gegessen.' => 길이: 18


### Autosizing Test

In [88]:
import win32com.client as win32

# PowerPoint AutoSize 상수
msoAutoSizeNone = 0              # 자동 크기 조정 안 함
msoAutoSizeShapeToFitText = 1    # 텍스트에 맞춰 도형 크기 조정
msoAutoSizeTextToFitShape = 2    # 도형 크기에 맞춰 텍스트(폰트) 크기 조정

def apply_autofit_to_shape(shape):
    """
    주어진 shape의 TextFrame2에 AutoSize를 적용하여
    텍스트가 도형을 넘어가면 폰트 크기가 줄어들도록 설정
    """
    if not shape.HasTextFrame:
        return

    tf2 = shape.TextFrame2

    # 텍스트 줄바꿈 켜기 (AutoSize가 제대로 동작하도록)
    tf2.WordWrap = -1  # msoTrue

    # 세로 정렬/여백은 선택 사항 (보기 좋게 세팅)
    tf2.VerticalAnchor = 3  # msoAnchorMiddle (가운데 정렬)
    tf2.MarginBottom = 5
    tf2.MarginTop = 5
    tf2.MarginLeft = 5
    tf2.MarginRight = 5

    # 핵심: 텍스트를 도형에 맞게 자동 축소
    tf2.AutoSize = msoAutoSizeTextToFitShape


def main():
    # PowerPoint 실행 또는 연결
    ppt = win32.gencache.EnsureDispatch("PowerPoint.Application")
    ppt.Visible = True

    # 새 프레젠테이션 생성
    presentation = ppt.Presentations.Add()

    # 빈 슬라이드 추가 (레이아웃 12: ppLayoutBlank)
    slide = presentation.Slides.Add(1, 12)

    # 도형 하나 추가 (텍스트 상자)
    left, top, width, height = 100, 100, 400, 150
    shape = slide.Shapes.AddShape(1, left, top, width, height)  # 1: msoShapeRectangle

    # 긴 텍스트 넣어서 넘치게 만들기
    long_text = (
        "이 텍스트는 win32com을 이용해 AutoSize 기능을 실험하기 위한 예제입니다. "
        "도형의 경계를 넘어가도록 충분히 긴 문장을 넣어두면, "
        "TextFrame2.AutoSize = TextToFitShape 설정에 의해 폰트 크기가 자동으로 줄어들게 됩니다. "
        "줄바꿈이 켜져 있는 상태에서 더 잘 동작합니다."
    )

    shape.TextFrame2.TextRange.Text = long_text

    # AutoSize 적용
    apply_autofit_to_shape(shape)

    print("실험용 슬라이드가 PowerPoint에 생성되었습니다!")

if __name__ == "__main__":
    main()

실험용 슬라이드가 PowerPoint에 생성되었습니다!


In [90]:

import win32com.client as win32

# PowerPoint AutoSize 상수
msoAutoSizeNone = 0              # 자동 크기 조정 안 함
msoAutoSizeShapeToFitText = 1    # 텍스트에 맞춰 도형 크기 조정
msoAutoSizeTextToFitShape = 2    # 도형 크기에 맞춰 텍스트(폰트) 크기 조정

# 기타 상수
msoTrue = -1
msoTextOrientationHorizontal = 1  # 가로 텍스트
msoShapeRectangle = 1             # (참고: 도형 사각형)
msoTextBox = 17                   # 텍스트 상자 (버전마다 다를 수 있음)

def apply_autofit_to_shape(shape):
    """
    주어진 shape의 TextFrame2에 AutoSize를 적용하여
    텍스트가 도형을 넘어가면 폰트 크기가 줄어들도록 설정
    (도형이든 일반 텍스트박스든 HasTextFrame이면 모두 적용 가능)
    """
    if not shape.HasTextFrame:
        return

    tf2 = shape.TextFrame2

    # 텍스트 줄바꿈 켜기 (AutoSize가 제대로 동작하도록)
    tf2.WordWrap = msoTrue

    # 세로 정렬/여백 (선택)
    tf2.VerticalAnchor = 3  # msoAnchorMiddle (가운데 정렬)
    tf2.MarginBottom = 5
    tf2.MarginTop = 5
    tf2.MarginLeft = 5
    tf2.MarginRight = 5

    # 핵심: 텍스트를 도형에 맞게 자동 축소
    tf2.AutoSize = msoAutoSizeTextToFitShape


def main():
    # PowerPoint 실행 또는 연결
    ppt = win32.gencache.EnsureDispatch("PowerPoint.Application")
    ppt.Visible = True

    # 새 프레젠테이션 생성
    presentation = ppt.Presentations.Add()

    # 빈 슬라이드 추가 (레이아웃 12: ppLayoutBlank)
    slide = presentation.Slides.Add(1, 12)

    left, top, width, height = 100, 100, 400, 150

    # 🔹 1) 일반 텍스트박스 생성 (AutoShape 말고 TextBox)
    textbox = slide.Shapes.AddTextbox(
        Orientation=msoTextOrientationHorizontal,
        Left=left,
        Top=top,
        Width=width,
        Height=height,
    )

    # 긴 텍스트 넣기
    long_text = (
        "이 텍스트는 win32com을 이용해 AutoSize 기능을 실험하기 위한 예제입니다. "
        "도형의 경계를 넘어가도록 충분히 긴 문장을 넣어두면, "
        "TextFrame2.AutoSize = TextToFitShape 설정에 의해 폰트 크기가 자동으로 줄어들게 됩니다. "
        "줄바꿈이 켜져 있는 상태에서 더 잘 동작합니다."
    )

    textbox.TextFrame2.TextRange.Text = long_text

    # 🔹 2) AutoSize 적용
    apply_autofit_to_shape(textbox)

    # 🔹 3) 텍스트에 취소선(strikethrough) 적용
    # TextRange 전체에 대해 폰트 속성 변경
    text_range = textbox.TextFrame2.TextRange
    text_range.Font.StrikeThrough = msoTrue

    print("일반 텍스트박스에 AutoSize + 취소선이 적용된 슬라이드가 생성되었습니다!")

if __name__ == "__main__":
    main()

일반 텍스트박스에 AutoSize + 취소선이 적용된 슬라이드가 생성되었습니다!


### TextFrame2

In [91]:
def inspect_object(obj):
    results = {}
    for attr in dir(obj):
        if attr.startswith('get_') or attr.startswith('_'): continue
        try:
            results[attr] = getattr(obj, attr)
        except:
            results[attr] = "<Unable to read>"
    return results

import pprint
pprint.pprint(inspect_object(shape.TextFrame2))

com_error: (-2147023174, 'The RPC server is unavailable.', None, None)

In [98]:
import win32com.client
import os

def create_and_verify_runs():
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    ppt_app.Visible = True
    
    presentation = ppt_app.Presentations.Add()
    slide = presentation.Slides.Add(1, 12) # 12: ppLayoutBlank
    
    shape = slide.Shapes.AddTextbox(1, 100, 100, 500, 200)
    tf2 = shape.TextFrame2
    tr2 = tf2.TextRange
    
    print("--- 1. 스타일이 적용된 Runs 생성 ---")
    
    # InsertAfter를 통해 텍스트를 추가하고 스타일 설정
    r1 = tr2.InsertAfter("Red Bold ")
    r1.Font.Bold = -1
    r1.Font.Fill.ForeColor.RGB = 0x0000FF
    
    r2 = tr2.InsertAfter("Blue Italic ")
    r2.Font.Italic = -1
    r2.Font.Fill.ForeColor.RGB = 0xFF0000
    
    r3 = tr2.InsertAfter("Normal")
    r3.Font.Spacing = 5

    print("--- 2. 생성된 데이터 검증 ---")

    # [수정 포인트] 
    # tr2.Runs()가 가끔 문자열을 반환하는 문제를 방지하기 위해 
    # 명시적으로 개별 Run에 접근하거나 Count를 가져옵니다.
    
    try:
        # Runs 메서드에 인자를 주지 않고 호출했을 때의 모호함을 피하기 위해
        # 개별 run의 개수를 가져오는 가장 안전한 방법입니다.
        runs_obj = tr2.Runs()
        
        # 만약 runs_obj가 문자열로 넘어왔다면, 
        # 이는 해당 TextRange에 스타일 변화가 없거나 win32com의 해석 오류입니다.
        if isinstance(runs_obj, str):
            print(f"전체 텍스트(단일 Run): {runs_obj}")
            run_count = 1
        else:
            run_count = runs_obj.Count

        for i in range(1, run_count + 1):
            # Item(i) 또는 Runs(i)를 통해 명시적으로 i번째 조각을 가져옵니다.
            run = tr2.Runs(i) 
            f = run.Font
            
            print(f"Run {i}: '{run.Text}'")
            print(f"  - Bold: {f.Bold}, Size: {f.Size}, Color: {hex(f.Fill.ForeColor.RGB)}")

    except Exception as e:
        print(f"상세 에러 발생: {e}")

    # AutoSize 설정
    tf2.AutoSize = 2 # msoAutoSizeTextToFitShape
    print("\n--- 3. AutoSize 설정 완료 ---")

if __name__ == "__main__":
    create_and_verify_runs()

--- 1. 스타일이 적용된 Runs 생성 ---
--- 2. 생성된 데이터 검증 ---
전체 텍스트(단일 Run): Red Bold Blue Italic Normal
상세 에러 발생: (-2147352559, 'Does not support a collection.', None, None)

--- 3. AutoSize 설정 완료 ---


In [100]:
import win32com.client
import json
import os

def get_run_style(run):
    """개별 Run 객체에서 스타일 정보를 추출합니다."""
    f = run.Font
    color = f.Fill.ForeColor.RGB
    # RGB 변환 (Long -> hex/rgb)
    r, g, b = color & 255, (color >> 8) & 255, (color >> 16) & 255
    
    return {
        "text": run.Text,
        "font_name": f.Name,
        "size": f.Size,
        "bold": f.Bold,     # -1: True, 0: False
        "italic": f.Italic,
        "color_rgb": [r, g, b],
        "spacing": f.Spacing,
        "underline": f.UnderlineStyle
    }

def parse_ppt_to_json(input_path, output_path):
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    # 파일 열기 (WithWindow=False로 백그라운드 작업 가능)
    presentation = ppt_app.Presentations.Open(os.path.abspath(input_path), WithWindow=True)
    
    full_data = {
        "file_name": os.path.basename(input_path),
        "slides": []
    }

    try:
        for i, slide in enumerate(presentation.Slides, start=1):
            slide_data = {"slide_index": i, "elements": []}
            
            for shape in slide.Shapes:
                # 텍스트가 있는 도형인지 확인
                if shape.HasTextFrame:
                    tf2 = shape.TextFrame2
                    if tf2.HasText:
                        element = {
                            "shape_name": shape.Name,
                            "shape_id": shape.Id,
                            "runs": []
                        }
                        
                        tr2 = tf2.TextRange
                        try:
                            runs_obj = tr2.Runs()
                            # win32com 특성상 스타일이 하나면 문자열로 반환되는 것 방어
                            run_count = 1 if isinstance(runs_obj, str) else runs_obj.Count
                            
                            for r_idx in range(1, run_count + 1):
                                run = tr2.Runs(r_idx)
                                element["runs"].append(get_run_style(run))
                                
                        except Exception as e:
                            print(f"Error in shape {shape.Name}: {e}")
                            
                        slide_data["elements"].append(element)
            
            full_data["slides"].append(slide_data)

        # JSON 저장
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(full_data, f, ensure_ascii=False, indent=4)
            
        print(f"Successfully saved to {output_path}")

    finally:
        presentation.Close()
        ppt_app.Quit()

if __name__ == "__main__":
    INPUT_FILE = r"C:\Users\ben81\Documents\EditPPT\sample_ppt\math_simple.pptx"  # 실제 파일명으로 수정
    OUTPUT_FILE = "ppt_structure.json"
    parse_ppt_to_json(INPUT_FILE, OUTPUT_FILE)

Error in shape 제목 1: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 부제목 2: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 제목 1: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 내용 개체 틀 2: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 슬라이드 번호 개체 틀 3: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 제목 1: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 내용 개체 틀 2: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 슬라이드 번호 개체 틀 3: (-2147352559, 'Does not support a collection.', None, None)
Error in shape TextBox 6: (-2147352559, 'Does not support a collection.', None, None)
Error in shape TextBox 7: (-2147352559, 'Does not support a collection.', None, None)
Error in shape TextBox 8: (-2147352559, 'Does not support a collection.', None, None)
Error in shape 제목 1: (-2147352559, 'Does not support a collecti

In [105]:
import win32com.client
import json
import os

def get_char_style(char_obj):
    """글자 한 개 혹은 범위의 스타일 정보를 추출"""
    f = char_obj.Font
    color = f.Fill.ForeColor.RGB
    r, g, b = color & 255, (color >> 8) & 255, (color >> 16) & 255
    
    return {
        "font_name": f.Name,
        "size": f.Size,
        "bold": f.Bold,
        "italic": f.Italic,
        "color_rgb": [r, g, b],
        "spacing": f.Spacing
    }

def is_same_style(s1, s2):
    """두 스타일이 동일한지 비교 (비교 항목은 필요에 따라 추가/제외)"""
    return (s1["font_name"] == s2["font_name"] and 
            s1["size"] == s2["size"] and 
            s1["bold"] == s2["bold"] and 
            s1["color_rgb"] == s2["color_rgb"])

def parse_ppt_to_json_scan(input_path, output_path):
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    presentation = ppt_app.Presentations.Open(os.path.abspath(input_path), WithWindow=True)
    
    full_data = {"file_name": os.path.basename(input_path), "slides": []}

    try:
        for slide in presentation.Slides:
            slide_data = {"slide_index": slide.SlideIndex, "elements": []}
            
            for shape in slide.Shapes:
                if shape.HasTextFrame and shape.TextFrame2.HasText:
                    tr = shape.TextFrame2.TextRange
                    total_len = tr.Length
                    if total_len == 0: continue

                    element = {"shape_name": shape.Name, "runs": []}
                    
                    # --- 스캐닝 로직 시작 ---
                    start_pos = 1
                    current_run_style = get_char_style(tr.Characters(1, 1))
                    
                    for i in range(2, total_len + 1):
                        char_style = get_char_style(tr.Characters(i, 1))
                        
                        # 스타일이 변하면 지금까지의 구간을 저장
                        if not is_same_style(current_run_style, char_style):
                            element["runs"].append({
                                "text": tr.Characters(start_pos, i - start_pos).Text,
                                "style": current_run_style
                            })
                            start_pos = i
                            current_run_style = char_style
                    
                    # 마지막 남은 구간 추가
                    element["runs"].append({
                        "text": tr.Characters(start_pos, total_len - start_pos + 1).Text,
                        "style": current_run_style
                    })
                    # ------------------------
                    
                    slide_data["elements"].append(element)
            
            full_data["slides"].append(slide_data)

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(full_data, f, ensure_ascii=False, indent=4)
            
    finally:
        presentation.Close()
        ppt_app.Quit()

if __name__ == "__main__":
    parse_ppt_to_json_scan(r"C:\Users\ben81\Documents\EditPPT\sample_ppt\math_simple.pptx", "ppt_structure.json")

com_error: (-2147352559, 'Does not support a collection.', None, None)

In [ ]:
Below is a mapping of the old runs' styles to the new text, preserving maximal style continuity. The approach is as follows:

- **Preserve math styling** for variables (𝑥, 𝑦, etc.), numbers, and operators where they appeared in the original.
- **Preserve bold** for key terms ("linear equation", "not linear", "Key point", etc.) as in the original.
- **Preserve font size** and color where possible, but adapt to English text.
- **Plain text** (explanatory, non-math) uses the default Calibri 24.0, as in the original.
- **Math text** uses Cambria Math, 20.0 or 24.0 as in the original, with italic for variables.

---

```json
[
  {"Text": "An equation in which all unknowns (usually ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 24.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ", ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "y", "Font": {"Name": "Cambria Math", "Size": 24.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ", etc.) have degree ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "1", "Font": {"Name": "Cambria Math", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " is called a ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "linear equation", "Font": {"Name": "Calibri", "Size": 24.0, "Bold": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ".\nExamples:\n", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  
  {"Text": "2", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "+3=0", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "\n", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "3", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "−5", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "y", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "+7=0", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "\nOn the other hand, the following are ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "not linear", "Font": {"Name": "Calibri", "Size": 24.0, "Bold": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ":\n", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  
  // x²+3=0 (degree 2)
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "²", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}}, 
  {"Text": "+3=0", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " (degree ", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "2", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ")\n", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  
  // xy+1=0 (multiplication xy appears)
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "y", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "+1=0", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " (multiplication ", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "xy", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " appears)\n", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  
  // 1/x+2=0 (x in the denominator)
  {"Text": "1", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "/", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "+2=0", "Font": {"Name": "Cambria Math", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " (", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "x", "Font": {"Name": "Cambria Math", "Size": 20.0, "Italic": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": " in the denominator)\n", "Font": {"Name": "Calibri", "Size": 20.0, "Color": {"R":0,"G":0,"B":0}}},
  
  // Key point
  {"Text": "👉 Key point: ", "Font": {"Name": "Calibri", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "A linear equation is one in which the unknowns are not multiplied or squared, and each unknown has degree ", "Font": {"Name": "Calibri", "Size": 24.0, "Bold": true, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": "1", "Font": {"Name": "Cambria Math", "Size": 24.0, "Color": {"R":0,"G":0,"B":0}}},
  {"Text": ".", "Font": {"Name": "Calibri", "Size": 24.0, "Bold": true, "Color": {"R":0,"G":0,"B":0}}}
]
```

---

**Notes:**
- Superscript ² is preserved as a separate run.
- Math variables (x, y, xy) are italic Cambria Math.
- Bold is used for "linear equation", "not linear", and the key point, matching the original's bold use.
- Font sizes are preserved as in the original (24 for main, 20 for math examples).
- All colors remain black as in the original.
- Line breaks (`\n`) are used as in the new text.

If you need the output in a different format or with further breakdown, let me know! 1인 식이 선형방정식.


### Overflow 확인

In [4]:
import win32com.client

# PowerPoint 실행
ppt = win32com.client.Dispatch("PowerPoint.Application")
ppt.Visible = True

# 새 프레젠테이션
pres = ppt.Presentations.Add()

# 빈 슬라이드 (레이아웃 6 = blank)
slide = pres.Slides.Add(1, 6)

# 텍스트 박스 추가
shape = slide.Shapes.AddTextbox(
    Orientation=1,  # msoTextOrientationHorizontal
    Left=100,
    Top=100,
    Width=300,
    Height=100,     # 일부러 작게
)

tf = shape.TextFrame
tf.WordWrap = True
tf.AutoSize = 0  # AutoSize 끔 (중요)

# overflow 유도용 긴 텍스트
tf.TextRange.Text = (
    "This is a very long text intended to overflow the text box. "
    "We want to verify that font size automatically shrinks "
    "until the text fits inside the shape. "
    "This should trigger BoundHeight overflow."
)

# 초기 폰트 크기 크게
tr = tf.TextRange
tr.Font.Size = 28

print("Initial font size:", tr.Font.Size)

# 수동 shrink 로직
while tr.BoundHeight > shape.Height and tr.Font.Size > 6:
    tr.Font.Size -= 0.5

print("Final font size:", tr.Font.Size)
print("Final BoundHeight:", tr.BoundHeight)
print("Shape Height:", shape.Height)


Initial font size: 28.0
Final font size: 8.0
Final BoundHeight: 28.799999237060547
Shape Height: 29.081260681152344


In [ ]:
chr(18))

18

In [4]:
ord('-')

45

In [5]:
import win32com.client

def create_bullet_test_slide():
    # 1. PowerPoint 앱 실행
    pp_app = win32com.client.Dispatch("PowerPoint.Application")
    pp_app.Visible = True
    
    # 2. 새 프레젠테이션 및 슬라이드 추가 (Layout 12는 Blank)
    prs = pp_app.Presentations.Add()
    slide = prs.Slides.Add(1, 12) 
    
    # 3. 텍스트 박스 추가 (Left, Top, Width, Height)
    textbox = slide.Shapes.AddTextbox(1, 50, 50, 500, 400)
    tf = textbox.TextFrame
    
    # --- 실험할 불렛 데이터 정의 ---
    # (텍스트, Type, CharacterCode)
    # Type 1: Unnumbered (기호), Type 2: Numbered (숫자)
    bullet_samples = [
        ("기본 점 불렛 (Dot)", 1, 8226),      # •
        ("대시 불렛 (Dash)", 1, 8211),      # –
        ("체크 불렛 (Check)", 1, 10004),    # ✔
        ("사각형 불렛 (Square)", 1, 9632),   # ■
        ("첫 번째 숫자 리스트", 2, None),    # 1.
        ("두 번째 숫자 리스트", 2, None),    # 2.
    ]

    for i, (text, b_type, b_char) in enumerate(bullet_samples):
        # 단락 추가
        para = tf.TextRange.Paragraphs(i + 1)
        para.Text = text + ("\r" if i < len(bullet_samples) - 1 else "")
        
        # 불렛 설정
        format = para.ParagraphFormat.Bullet
        format.Visible = True
        format.Type = b_type
        
        if b_type == 1: # 기호 불렛일 때만 Character 설정
            format.Character = b_char
            
        # 추가 스타일링: 크기 및 색상 (선택 사항)
        format.RelativeSize = 1.2  # 텍스트 대비 120% 크기
        para.Font.Size = 24

    print("불렛 실험 슬라이드 생성 완료!")

if __name__ == "__main__":
    create_bullet_test_slide()

불렛 실험 슬라이드 생성 완료!


In [8]:
import win32com.client

def read_bullet_detail_info():
    pp_app = win32com.client.GetActiveObject("PowerPoint.Application")
    slide = pp_app.ActiveWindow.View.Slide
    shape = slide.Shapes(1)
    
    # 숫자 스타일 매핑 사전 (자주 쓰이는 것들)
    # Reference: PpBulletCharacterType Enumeration
    style_map = {
        0: "Mixed",
        1: "AlphaLCPeriod (a. b. c.)",
        2: "AlphaUCPeriod (A. B. C.)",
        3: "ArabicParenRight (1) 2) 3))",
        4: "ArabicPeriod (1. 2. 3.)",
        5: "RomanLCParenBoth ((i))",
        6: "RomanLCPeriod (i. ii. iii.)",
        7: "RomanUCPeriod (I. II. III.)",
        # 필요에 따라 추가 가능
    }

    paragraphs = shape.TextFrame.TextRange.Paragraphs()
    
    for i, para in enumerate(paragraphs, start=1):
        bullet = para.ParagraphFormat.Bullet
        b_type = bullet.Type
        
        print(f"Paragraph {i}: '{para.Text.strip()}'")
        
        if b_type == 1:  # Symbol
            print(f"  - Type: Symbol (기호)")
            print(f"  - Char: {chr(bullet.Character)} ({bullet.Character})")
        
        elif b_type == 2:  # Numbered
            print(f"  - Type: Numbered (숫자)")
            # .Style 속성을 통해 어떤 숫자인지 확인
            style_code = bullet.Style
            style_name = style_map.get(style_code, f"Other Style({style_code})")
            print(f"  - Number Style: {style_name}")
            print(f"  - Start At: {bullet.StartValue}") # 시작 번호 (보통 1)
            
        else:
            print(f"  - Type: None")
        print("-" * 40)

if __name__ == "__main__":
    read_bullet_detail_info()

Paragraph 1: '기본 점 불렛 (Dot)'
  - Type: Symbol (기호)
  - Char: • (8226)
----------------------------------------
Paragraph 2: '대시 불렛 (Dash)'
  - Type: Symbol (기호)
  - Char: – (8211)
----------------------------------------
Paragraph 3: '체크 불렛 (Check)'
  - Type: Symbol (기호)
  - Char: ✔ (10004)
----------------------------------------
Paragraph 4: '사각형 불렛 (Square)'
  - Type: Symbol (기호)
  - Char: ■ (9632)
----------------------------------------
Paragraph 5: '첫 번째 숫자 리스트'
  - Type: Numbered (숫자)
  - Number Style: ArabicParenRight (1) 2) 3))
  - Start At: 1
----------------------------------------
Paragraph 6: '두 번째 숫자 리스트'
  - Type: Numbered (숫자)
  - Number Style: ArabicParenRight (1) 2) 3))
  - Start At: 1
----------------------------------------
